# Confident

>[DeepEval](https://confident-ai.com) 是用于LLM单元测试的包。
> 使用Confident，每个人都可以通过更快的迭代构建强大的语言模型，
> 同时利用单元测试和集成测试。我们为迭代过程中的每个步骤提供支持，
> 从合成数据创建到测试。


在本指南中，我们将演示如何测试和衡量LLM的性能。我们展示如何使用我们的回调来测量性能，以及如何定义自己的指标并将其记录到我们的仪表板中。

DeepEval还提供：
- 如何生成合成数据
- 如何测量性能
- 一个用于监控和随时间查看结果的仪表板

## 安装和设置

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai langchain-community deepeval langchain-chroma

### 获取API凭证

要获取DeepEval API凭证，请按照以下步骤操作：

1. 访问 https://app.confident-ai.com
2. 点击"Organization"
3. 复制API密钥。


登录时，您还将被要求设置`implementation`名称。实现名称是必需的，用于描述实现类型。（考虑您想为项目起什么名字。我们建议使其具有描述性。）

In [11]:
!deepeval login

### 设置DeepEval

默认情况下，您可以使用`DeepEvalCallbackHandler`来设置要跟踪的指标。但是，目前对指标的支持有限（很快会添加更多）。目前支持：
- [回答相关性](https://docs.confident-ai.com/docs/measuring_llm_performance/answer_relevancy)
- [偏见](https://docs.confident-ai.com/docs/measuring_llm_performance/debias)
- [毒性](https://docs.confident-ai.com/docs/measuring_llm_performance/non_toxic)

In [ ]:
from deepeval.metrics.answer_relevancy import AnswerRelevancy

# 在这里，我们希望确保答案至少是相关的
answer_relevancy_metric = AnswerRelevancy(minimum_score=0.5)

## 入门

要使用`DeepEvalCallbackHandler`，我们需要`implementation_name`。

In [ ]:
from langchain_community.callbacks.confident_callback import DeepEvalCallbackHandler

deepeval_callback = DeepEvalCallbackHandler(
    implementation_name="langchainQuickstart", metrics=[answer_relevancy_metric]
)

### 场景1：输入到LLM

然后您可以将其输入到您的OpenAI LLM中。

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0,
    callbacks=[deepeval_callback],
    verbose=True,
    openai_api_key="<YOUR_API_KEY>",
)
output = llm.generate(
    [
        "什么是最好的评估工具？（完全没有偏见）",
    ]
)

LLMResult(generations=[[Generation(text='\n\nQ: What did the fish say when he hit the wall? \nA: Dam.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe Moon \n\nThe moon is high in the midnight sky,\nSparkling like a star above.\nThe night so peaceful, so serene,\nFilling up the air with love.\n\nEver changing and renewing,\nA never-ending light of grace.\nThe moon remains a constant view,\nA reminder of life’s gentle pace.\n\nThrough time and space it guides us on,\nA never-fading beacon of hope.\nThe moon shines down on us all,\nAs it continues to rise and elope.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nQ. What did one magnet say to the other magnet?\nA. "I find you very attractive!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text="\n\nThe world is charged with the grandeur of God.\nIt will flame out, like shining from shook foil;\nIt gathers to a greatness, like t

然后，您可以通过调用`is_successful()`方法检查指标是否成功。

In [ ]:
answer_relevancy_metric.is_successful()
# 返回True/False

运行完成后，您应该能够在下方看到我们的仪表板。

![仪表板](https://docs.confident-ai.com/assets/images/dashboard-screenshot-b02db73008213a211b1158ff052d969e.png)

### 场景2：在没有回调的链中跟踪LLM

要在没有回调的链中跟踪LLM，您可以在最后插入它。

我们可以先定义一个简单的链，如下所示。

In [ ]:
import requests
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_file_url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"

openai_api_key = "sk-XXX"

with open("state_of_the_union.txt", "w") as f:
    response = requests.get(text_file_url)
    f.write(response.text)

loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=openai_api_key),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
)

# 提供一个新的问答管道
query = "谁是总统？"
result = qa.run(query)

定义链后，您可以手动检查答案相似性。

In [ ]:
answer_relevancy_metric.measure(result, query)
answer_relevancy_metric.is_successful()

### 下一步？

您可以在[这里](https://docs.confident-ai.com/docs/quickstart/custom-metrics)创建自己的自定义指标。

DeepEval还提供其他功能，如[自动创建单元测试](https://docs.confident-ai.com/docs/quickstart/synthetic-data-creation)、[幻觉测试](https://docs.confident-ai.com/docs/measuring_llm_performance/factual_consistency)。

如果您有兴趣，请查看我们的Github仓库[https://github.com/confident-ai/deepeval](https://github.com/confident-ai/deepeval)。我们欢迎任何PR和关于如何改进LLM性能的讨论。